<a href="https://colab.research.google.com/github/sachincs3108/Hands-on_project/blob/master/Classification_using_ML_Pipeline_with_PySpark_in_Databricks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deepali Handgar
## Roll No: 04
## Data Engineering Assignment: ML Pipeline for Classification
## M.Sc. Computer Science (With Specialization in Data Science)
## University of Mumbai

In [ ]:
# load data 
df_ipl = spark.read.option("inferSchema", True).option("header",True).csv("/FileStore/tables/ind_ban.csv")

In [ ]:
# view data
df_ipl.show()

+-------+-----------------+------+------------------+--------------------+------+---------+---+------+----------+--------+----+----+-------------------+
|Batsman|     Batsman_Name|Bowler|       Bowler_Name|          Commentary|Detail|Dismissed| Id|Isball|Isboundary|Iswicket|Over|Runs|          Timestamp|
+-------+-----------------+------+------------------+--------------------+------+---------+---+------+----------+--------+----+----+-------------------+
|  28994|   Mohammed Shami| 63881| Mustafizur Rahman|OUT! Bowled! 5-fe...|     W|    28994|346|  true|      null|       1|49.6|   0|2019-07-02 13:18:47|
|   5132|Bhuvneshwar Kumar| 63881| Mustafizur Rahman|WIDE AND RUN OUT!...|  W+wd|     5132|344|  true|      null|       1|49.6|   1|2019-07-02 13:17:28|
|  28994|   Mohammed Shami| 63881| Mustafizur Rahman|Back of a length ...|  null|     null|343|  true|      null|    null|49.5|   1|2019-07-02 13:16:03|
|   5132|Bhuvneshwar Kumar| 63881| Mustafizur Rahman|Just 1 run off th...|  null| 

In [ ]:
df_ipl.printSchema()

root
 |-- Batsman: integer (nullable = true)
 |-- Batsman_Name: string (nullable = true)
 |-- Bowler: integer (nullable = true)
 |-- Bowler_Name: string (nullable = true)
 |-- Commentary: string (nullable = true)
 |-- Detail: string (nullable = true)
 |-- Dismissed: integer (nullable = true)
 |-- Id: integer (nullable = true)
 |-- Isball: boolean (nullable = true)
 |-- Isboundary: integer (nullable = true)
 |-- Iswicket: integer (nullable = true)
 |-- Over: double (nullable = true)
 |-- Runs: integer (nullable = true)
 |-- Timestamp: timestamp (nullable = true)



In [ ]:
# map string columns to numeric indices
from pyspark.ml.feature import StringIndexer
df_ipl_mod = StringIndexer(inputCol="Batsman_Name",outputCol = 'bt_index').fit(df_ipl).transform(df_ipl)

In [ ]:
df_ipl_mod.select("Batsman_name","bt_index").show()

+-----------------+--------+
|     Batsman_name|bt_index|
+-----------------+--------+
|   Mohammed Shami|    18.0|
|Bhuvneshwar Kumar|    16.0|
|   Mohammed Shami|    18.0|
|Bhuvneshwar Kumar|    16.0|
|         MS Dhoni|     7.0|
|         MS Dhoni|     7.0|
|         MS Dhoni|     7.0|
|         MS Dhoni|     7.0|
|         MS Dhoni|     7.0|
|         MS Dhoni|     7.0|
|         MS Dhoni|     7.0|
|         MS Dhoni|     7.0|
|         MS Dhoni|     7.0|
|Bhuvneshwar Kumar|    16.0|
|         MS Dhoni|     7.0|
|         MS Dhoni|     7.0|
|Bhuvneshwar Kumar|    16.0|
|   Dinesh Karthik|    13.0|
|         MS Dhoni|     7.0|
|         MS Dhoni|     7.0|
+-----------------+--------+
only showing top 20 rows



In [ ]:
df_ipl.columns

Out[33]: ['Batsman',
 'Batsman_Name',
 'Bowler',
 'Bowler_Name',
 'Commentary',
 'Detail',
 'Dismissed',
 'Id',
 'Isball',
 'Isboundary',
 'Iswicket',
 'Over',
 'Runs',
 'Timestamp']

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, VectorAssembler

In [ ]:
df_ipl = df_ipl.drop(*['Batsman','Bowler','Id'])
stage2 = StringIndexer(inputCol= 'Batsman_Name', outputCol = 'bm_index')
stage3 = StringIndexer(inputCol= 'Bowler_Name', outputCol = 'bowler_index')
stage4 = OneHotEncoder(inputCols = ['bm_index', 'bowler_index'], outputCols = ['bm_ohe', 'bowler_ohe'])

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-4264977406128474> in <module>
----> 1 stage1.show()

NameError: name 'stage1' is not defined

In [ ]:
# combine list of columns into a single vector
stage5 = VectorAssembler(inputCols = ['Dismissed','Isball','Isboundary','Over','Runs','bm_ohe','bowler_ohe'], outputCol = 'features')

In [ ]:
from pyspark.ml.classification import LogisticRegression
stage6 = LogisticRegression(featuresCol = 'features', labelCol = 'Iswicket')

In [ ]:
# Pipeline
pl = Pipeline(stages = [stage2,stage3,stage4,stage5,stage6])

In [ ]:
df_ipl = df_ipl.fillna(0)

In [ ]:
logit_mod = pl.fit(df_ipl).transform(df_ipl)

Exception ignored in: <function JavaWrapper.__del__ at 0x7f465e48f040>
Traceback (most recent call last):
  File "/databricks/spark/python/pyspark/ml/wrapper.py", line 39, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'LogisticRegression' object has no attribute '_java_obj'
Exception ignored in: <function JavaWrapper.__del__ at 0x7f465e48f040>
Traceback (most recent call last):
  File "/databricks/spark/python/pyspark/ml/wrapper.py", line 39, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'LogisticRegression' object has no attribute '_java_obj'


In [ ]:
logit_mod.select('features','Iswicket','prediction').show()

+--------------------+--------+----------+
|            features|Iswicket|prediction|
+--------------------+--------+----------+
|(35,[0,1,3,23,24]...|       1|       1.0|
|(35,[0,1,3,4,21,2...|       1|       1.0|
|(35,[1,3,4,23,24]...|       0|       0.0|
|(35,[1,3,4,21,24]...|       0|       0.0|
|(35,[0,1,3,12,24]...|       1|       1.0|
|(35,[1,3,12,24],[...|       0|       0.0|
|(35,[1,3,12,24],[...|       0|       0.0|
|(35,[1,3,4,12,32]...|       0|       0.0|
|(35,[1,2,3,4,12,3...|       0|       0.0|
|(35,[1,3,12,32],[...|       0|       0.0|
|(35,[1,3,12,32],[...|       0|       0.0|
|(35,[1,2,3,4,12,3...|       0|       0.0|
|(35,[1,3,4,12,32]...|       0|       0.0|
|(35,[1,3,21,24],[...|       0|       0.0|
|(35,[1,3,4,12,24]...|       0|       0.0|
|(35,[1,3,12,24],[...|       0|       0.0|
|(35,[1,3,4,21,24]...|       0|       0.0|
|(35,[0,1,3,18,24]...|       1|       1.0|
|(35,[1,3,4,12,24]...|       0|       0.0|
|(35,[1,3,4,12,32]...|       0|       0.0|
+----------

In [ ]:
logit_mod.columns

Out[51]: ['Batsman_Name',
 'Bowler_Name',
 'Commentary',
 'Detail',
 'Dismissed',
 'Isball',
 'Isboundary',
 'Iswicket',
 'Over',
 'Runs',
 'Timestamp',
 'bm_index',
 'bowler_index',
 'bm_ohe',
 'bowler_ohe',
 'features',
 'rawPrediction',
 'probability',
 'prediction']

In [ ]:
# splitting data into train and test

In [ ]:
stage2 = StringIndexer(inputCol = 'Batsman_Name', outputCol = 'bm_index')
stage3 = StringIndexer(inputCol = 'Bowler_Name', outputCol = 'bowler_index')
stage4 = OneHotEncoder(inputCols = ['bm_index','bowler_index'], outputCols = ['bm_ohe','bowler_ohe'])

In [ ]:
pl1 = Pipeline(stages = [stage2,stage3,stage4])

In [ ]:
df_mod = pl1.fit(df_ipl).transform(df_ipl)

In [ ]:
df_mod.show()

+-----------------+------------------+--------------------+------+---------+------+----------+--------+----+----+-------------------+--------+------------+---------------+--------------+
|     Batsman_Name|       Bowler_Name|          Commentary|Detail|Dismissed|Isball|Isboundary|Iswicket|Over|Runs|          Timestamp|bm_index|bowler_index|         bm_ohe|    bowler_ohe|
+-----------------+------------------+--------------------+------+---------+------+----------+--------+----+----+-------------------+--------+------------+---------------+--------------+
|   Mohammed Shami| Mustafizur Rahman|OUT! Bowled! 5-fe...|     W|    28994|  true|         0|       1|49.6|   0|2019-07-02 13:18:47|    18.0|         0.0|(19,[18],[1.0])|(11,[0],[1.0])|
|Bhuvneshwar Kumar| Mustafizur Rahman|WIDE AND RUN OUT!...|  W+wd|     5132|  true|         0|       1|49.6|   1|2019-07-02 13:17:28|    16.0|         0.0|(19,[16],[1.0])|(11,[0],[1.0])|
|   Mohammed Shami| Mustafizur Rahman|Back of a length ...|  null

In [ ]:
df_vec = VectorAssembler(inputCols = ['Dismissed','Isball','Isboundary','Over','Runs','bm_ohe','bowler_ohe'],outputCol = 'features').transform(df_mod)

In [ ]:
df_vec.show()

+-----------------+------------------+--------------------+------+---------+------+----------+--------+----+----+-------------------+--------+------------+---------------+--------------+--------------------+
|     Batsman_Name|       Bowler_Name|          Commentary|Detail|Dismissed|Isball|Isboundary|Iswicket|Over|Runs|          Timestamp|bm_index|bowler_index|         bm_ohe|    bowler_ohe|            features|
+-----------------+------------------+--------------------+------+---------+------+----------+--------+----+----+-------------------+--------+------------+---------------+--------------+--------------------+
|   Mohammed Shami| Mustafizur Rahman|OUT! Bowled! 5-fe...|     W|    28994|  true|         0|       1|49.6|   0|2019-07-02 13:18:47|    18.0|         0.0|(19,[18],[1.0])|(11,[0],[1.0])|(35,[0,1,3,23,24]...|
|Bhuvneshwar Kumar| Mustafizur Rahman|WIDE AND RUN OUT!...|  W+wd|     5132|  true|         0|       1|49.6|   1|2019-07-02 13:17:28|    16.0|         0.0|(19,[16],[1.0

In [ ]:
df_vector = df_vec.select('features','Iswicket')

In [ ]:
splits = df_vector.randomSplit([0.7,0.3])

In [ ]:
df_train = splits[0]

In [ ]:
df_test = splits[1]

In [ ]:
df_train.show(10)

+--------------------+--------+
|            features|Iswicket|
+--------------------+--------+
|(35,[0,1,3,4,21,2...|       1|
|(35,[0,1,3,5,33],...|       1|
|(35,[0,1,3,6,31],...|       1|
|(35,[0,1,3,8,28],...|       1|
|(35,[0,1,3,10,26]...|       1|
|(35,[0,1,3,11,25]...|       1|
|(35,[0,1,3,13,30]...|       1|
|(35,[0,1,3,14,24]...|       1|
|(35,[0,1,3,15,25]...|       1|
|(35,[0,1,3,18,24]...|       1|
+--------------------+--------+
only showing top 10 rows



In [ ]:
logit_mod_tr = LogisticRegression(featuresCol = 'features',labelCol = 'Iswicket').fit(df_train).transform(df_train)

In [ ]:
logit_mod_test = LogisticRegression(featuresCol = 'features',labelCol = 'Iswicket').fit(df_train).transform(df_test)

Exception ignored in: <function JavaWrapper.__del__ at 0x7f465e48f040>
Traceback (most recent call last):
  File "/databricks/spark/python/pyspark/ml/wrapper.py", line 39, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'OneHotEncoder' object has no attribute '_java_obj'


In [ ]:
logit_mod_test.select(['features','Iswicket','prediction']).show()

+--------------------+--------+----------+
|            features|Iswicket|prediction|
+--------------------+--------+----------+
|(35,[0,1,3,7,25],...|       1|       0.0|
|(35,[0,1,3,12,24]...|       1|       0.0|
|(35,[0,1,3,16,27]...|       1|       0.0|
|(35,[0,1,3,17,26]...|       1|       0.0|
|(35,[0,1,3,19,26]...|       1|       1.0|
|(35,[0,1,3,22,24]...|       1|       1.0|
|(35,[0,1,3,23,24]...|       1|       1.0|
|(35,[0,1,3,26],[6...|       1|       1.0|
|(35,[1,2,3,4,5],[...|       0|       0.0|
|(35,[1,2,3,4,5,24...|       0|       0.0|
|(35,[1,2,3,4,5,24...|       0|       0.0|
|(35,[1,2,3,4,5,34...|       0|       0.0|
|(35,[1,2,3,4,5,34...|       0|       0.0|
|(35,[1,2,3,4,6,31...|       0|       0.0|
|(35,[1,2,3,4,6,34...|       0|       0.0|
|(35,[1,2,3,4,7,25...|       0|       0.0|
|(35,[1,2,3,4,7,26...|       0|       0.0|
|(35,[1,2,3,4,8],[...|       0|       0.0|
|(35,[1,2,3,4,9,25...|       0|       0.0|
|(35,[1,2,3,4,9,26...|       0|       0.0|
+----------

In [ ]:
# Evaluation
from pyspark.ml.evaluation import BinaryClassificationEvaluator
eval = BinaryClassificationEvaluator(labelCol = 'Iswicket', rawPredictionCol = 'prediction', metricName= 'areaUnderROC')

In [ ]:
eval.evaluate(logit_mod_test)

Out[76]: 0.7471751412429378

In [ ]:
eval.evaluate(logit_mod)

Out[77]: 1.0